načtení balíčků

In [81]:
from google.cloud import language_v1
from google.oauth2 import service_account
import pandas as pd

připojení google API

In [82]:
creds = service_account.Credentials.from_service_account_file('./credentials.json')

In [83]:
client = language_v1.LanguageServiceClient(credentials=creds)

In [84]:
text = '''Google, headquartered in Mountain View (1600 Amphitheatre Pkwy, Mountain View, CA 940430),
unveiled the new Android phone for $799 at the Consumer Electronic Show. Sundar Pichai said in his keynote
that users love their new Android phones.'''

In [85]:
language = "en"
type_ = language_v1.Document.Type.PLAIN_TEXT
encoding_type = language_v1.EncodingType.UTF8
document = {"content": text, "type_": type_, "language": language}

In [86]:
response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})

In [87]:
score = response.document_sentiment.score
magnitude = response.document_sentiment.magnitude
print("Overall Sentiment: score of {} with magnitude of {}".format(score, magnitude))

Overall Sentiment: score of 0.20000000298023224 with magnitude of 0.5


In [88]:
def analyze_entity(text):
    language = "en"
    type_ = language_v1.Document.Type.PLAIN_TEXT
    encoding_type = language_v1.EncodingType.UTF8
    document = {"content": text, "type_": type_, "language": language}
    response = client.analyze_entities(request = {'document': document, 'encoding_type': encoding_type})
    return response

In [89]:
print(text)
response = analyze_entity(text)

Google, headquartered in Mountain View (1600 Amphitheatre Pkwy, Mountain View, CA 940430),
unveiled the new Android phone for $799 at the Consumer Electronic Show. Sundar Pichai said in his keynote
that users love their new Android phones.


Načtení datasetu

In [90]:
df = pd.read_csv("hotel_reviews.csv")

In [91]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


Samplování a úprava recenzí

In [98]:
df = df.sample(frac = 0.2, replace = False, random_state=42)

In [93]:
df["text"] = df["Negative_Review"] + df["Positive_Review"]

In [94]:
df["text"] = df["text"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [99]:
text_content = df['text'].values[50]
print(text_content)

 My room was on two levels with stairs could be a prob if you had mobility issues could possibly be noisy as window looked down on the street was ok So no biggie Located almost opposite the main station Clean tidy Loved the shower


In [102]:
response = analyze_entity(text_content)

In [103]:
for entity in response.entities:
    print(u"Název entity: {} - Typ entity: {} - Skóre: {}".format(
        entity.name,
        language_v1.Entity.Type(entity.type_).name,
        entity.salience))

Název entity: room - Typ entity: LOCATION - Skóre: 0.2515648305416107
Název entity: levels - Typ entity: OTHER - Skóre: 0.22605377435684204
Název entity: prob - Typ entity: OTHER - Skóre: 0.1128593161702156
Název entity: stairs - Typ entity: OTHER - Skóre: 0.0926283597946167
Název entity: mobility issues - Typ entity: OTHER - Skóre: 0.07767260074615479
Název entity: window - Typ entity: OTHER - Skóre: 0.0627623200416565
Název entity: street - Typ entity: LOCATION - Skóre: 0.059135980904102325
Název entity: station - Typ entity: LOCATION - Skóre: 0.04490835219621658
Název entity: shower - Typ entity: OTHER - Skóre: 0.03362822160124779
Název entity: biggie - Typ entity: OTHER - Skóre: 0.0199526809155941
Název entity: Clean - Typ entity: PERSON - Skóre: 0.018833566457033157
Název entity: two - Typ entity: NUMBER - Skóre: 0.0
